In [28]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from cvxopt import matrix 
from cvxopt import solvers

df_train = pd.read_csv("train.csv")
test_set  = pd.read_csv("test.csv")

training_set = df_train.iloc[:4000]
validation_set = df_train.iloc[4000:]

In [29]:
def split(data):
    y = data.iloc[:, 0].replace(0, -1).to_numpy().astype(float)
    X = data.iloc[:, 1:].to_numpy().astype(float)
    return y, X

In [30]:
def linear_kernel(X, Z):
    return X @ Z.T

def polynomial_kernel(X, Z, C, D):
    return (X @ Z.T + C)**D 

def gaussian_kernel(X, Y, sigma):
    return np.exp((-linalg.norm(x-y)**2)/(2*(sigma**2)))

def rbf_kernel(X, Z, gamma):
    X2 = (X**2).sum(axis=1, keepdims=True)    
    Z2 = (Z**2).sum(axis=1, keepdims=True).T  
    d2 = X2 + Z2 - 2.0 * (X @ Z.T)
    return np.exp(-gamma * np.clip(d2, 0.0, None))

In [31]:
y_train, X_train = split(training_set)
y_validation, X_validation = split(validation_set)
y_test, X_test = split(test_set)

scaler = StandardScaler()   

X_train_scaled = scaler.fit_transform(X_train)   
X_val_scaled   = scaler.transform(X_validation)
X_test_scaled  = scaler.transform(X_test)

m = X_train_scaled.shape[0] 
y = y_train.reshape(-1, 1).astype(float)

K = linear_kernel(X_train_scaled, X_train_scaled)
P = matrix(np.outer(y, y) * K)

q = matrix(-np.ones((m, 1)))
C = 1.0 
G = matrix(np.vstack([-np.eye(m), np.eye(m)]))
h = matrix(np.hstack([np.zeros(m), C * np.ones(m)]))

A = matrix(y.reshape(1, -1))
b = matrix(0.0)

solution = solvers.qp(P,q,G,h,A,b)

a = np.ravel(solution['x'])


sv = a > 1e-5
ind = np.arange(len(a))[sv]
a_sv = a[sv]
X_sv = X_train_scaled[sv]
y_sv = y_train[sv]

b = 0
for n in range(len(a_sv)):
    b += y_sv[n]
    b -= np.sum(a_sv * y_sv * K[ind[n], sv])
b /= len(a_sv)

     pcost       dcost       gap    pres   dres
 0: -7.4843e+02 -1.0915e+04  7e+04  4e+00  5e-13
 1: -4.6207e+02 -7.0177e+03  1e+04  5e-01  3e-13
 2: -2.9736e+02 -2.1717e+03  3e+03  1e-01  2e-13
 3: -2.3142e+02 -1.0201e+03  1e+03  4e-02  2e-13
 4: -2.0123e+02 -5.5389e+02  5e+02  1e-02  1e-13
 5: -1.9467e+02 -3.3695e+02  2e+02  5e-03  1e-13
 6: -1.9620e+02 -2.5783e+02  7e+01  1e-03  2e-13
 7: -2.0014e+02 -2.3536e+02  4e+01  5e-04  1e-13
 8: -2.0281e+02 -2.2359e+02  2e+01  2e-04  2e-13
 9: -2.0645e+02 -2.1524e+02  9e+00  5e-05  2e-13
10: -2.0734e+02 -2.1319e+02  6e+00  2e-05  1e-13
11: -2.0838e+02 -2.1142e+02  3e+00  6e-06  2e-13
12: -2.0875e+02 -2.1083e+02  2e+00  3e-06  2e-13
13: -2.0940e+02 -2.0998e+02  6e-01  5e-07  2e-13
14: -2.0953e+02 -2.0981e+02  3e-01  1e-14  2e-13
15: -2.0964e+02 -2.0969e+02  5e-02  4e-15  2e-13
16: -2.0966e+02 -2.0967e+02  3e-03  3e-14  2e-13
17: -2.0966e+02 -2.0966e+02  8e-05  6e-14  2e-13
Optimal solution found.


In [32]:
def decision_function(X_new):
    K_new = X_new @ X_sv.T
    return (a_sv * y_sv) @ K_new.T + b

def predict(X_new):
    return np.sign(decision_function(X_new))

In [34]:
y_pred_train = predict(X_train_scaled)
y_pred_val   = predict(X_val_scaled)

train_acc = (y_pred_train == y_train).mean()
val_acc   = (y_pred_val == y_validation).mean()

print(f"Train acc: {train_acc:.4f}, Val acc: {val_acc:.4f}")

Train acc: 0.9818, Val acc: 0.9624
